In [1]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)


2.8.0


In [2]:
# def convnet_model_():
#     vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
#     vgg.trainable = False
#     model = tf.keras.Sequential([
#         #VGG16
#         vgg,
#         #GlobalAveragePooling2D
#         tf.keras.layers.GlobalAveragePooling2D(),
#         #Dense 4096 relu
#         tf.keras.layers.Dense(4096, activation='relu'),
#         # #Dropout 0.5
#         # tf.keras.layers.Dropout(0.5),
#         # #Dense 4096 relu
#         # tf.keras.layers.Dense(4096, activation='relu'),
#         #Dropout 0.5
#         tf.keras.layers.Dropout(0.5),
#         #L2 normalization
#         tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
#     ])

#     return model

def convnet_model_():
    URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
    feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224,3))
    feature_extractor.trainable = False
    model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        #L2 normalization
        tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
    ])

    return model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(2048)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


In [3]:
deep_rank_model = deep_rank_model()

# for layer in deep_rank_model.layers:
#     print (layer.name, layer.output_shape)

model_path = "./deep_ranking"

deep_rank_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 96)   18528       ['input_1[0][0]']                
                                                                                              

In [4]:
# tf.saved_model.load(deep_rank_model, "./deep_ranking")
deep_rank_model.load_weights('deepranking')


# test set predictions

In [5]:
image1 = np.array(Image.open("food_224/00000.jpg"))
image1 = image1[None]
for i in range(5):
    pred = deep_rank_model.predict([image1, image1, image1])[0]
    print(pred.shape)

(2048,)
(2048,)
(2048,)
(2048,)
(2048,)


In [6]:
# dictionary with all the image names and their features
features_dict = {}

#images (warning this takes about 15GB of RAM)
images = np.zeros((10000,224,224,3))

# convert every image into features
line_id = 0
for filename in os.listdir('food_224'):
    img = np.array(Image.open('food_224/' + filename))/255
    images[line_id] = img
    line_id += 1
    if line_id % 10000 == 0:
        print(line_id)
        break
    
print(images.shape)



10000
(10000, 224, 224, 3)


In [7]:
size = 100

features = deep_rank_model.predict([images[0:size], images[0:size], images[0:size]])

for i in np.arange(size,10001-size,size):
    print("predicting",i)
    f = deep_rank_model.predict([images[i:i+size], images[i:i+size], images[i:i+size]])
    features = np.concatenate([features, f])


line_id = 0
for filename in os.listdir('food_224'):
    features_dict[filename.replace(".jpg","")] = features[line_id]
    line_id += 1
    if line_id % 10000 == 0:
        print(line_id)
        break




predicting 100
predicting 200
predicting 300
predicting 400
predicting 500
predicting 600
predicting 700
predicting 800
predicting 900
predicting 1000
predicting 1100
predicting 1200
predicting 1300
predicting 1400
predicting 1500
predicting 1600
predicting 1700
predicting 1800
predicting 1900
predicting 2000
predicting 2100
predicting 2200
predicting 2300
predicting 2400
predicting 2500
predicting 2600
predicting 2700
predicting 2800
predicting 2900
predicting 3000
predicting 3100
predicting 3200
predicting 3300
predicting 3400
predicting 3500
predicting 3600
predicting 3700
predicting 3800
predicting 3900
predicting 4000
predicting 4100
predicting 4200
predicting 4300
predicting 4400
predicting 4500
predicting 4600
predicting 4700
predicting 4800
predicting 4900
predicting 5000
predicting 5100
predicting 5200
predicting 5300
predicting 5400
predicting 5500
predicting 5600
predicting 5700
predicting 5800
predicting 5900
predicting 6000
predicting 6100
predicting 6200
predicting 6300
p

In [8]:
print("saving features")

# save the features dictionary to a file using pickle
with open('features.pickle', 'wb') as handle:
    pickle.dump(features_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


print(features_dict['00001'].shape)
# print(features_dict)

saving features
(2048,)


In [9]:

features_dict = {}
# load the features dictionary from the file
with open('features.pickle', 'rb') as handle:
    features_dict = pickle.load(handle)
    
print(features_dict['00001'].shape)

(2048,)


In [13]:

pred = []
line_id = 0
with open('test_triplets.txt', 'r') as f:
    # go through the lines
    for line in f:
        line = line.replace("\n", "")


        embedding1 = features_dict[line.split(' ')[0]]
        embedding2 = features_dict[line.split(' ')[1]]
        embedding3 = features_dict[line.split(' ')[2]]

        
        #compare distance between embeddings
        distance1 = np.sqrt(np.sum((embedding1 - embedding2)**2))
        distance2 = np.sqrt(np.sum((embedding1 - embedding3)**2))

        pred.append(1 if distance1 < distance2 else 0)

        line_id += 1
        if line_id % 1000 == 0:
            print(line_id)
            # break

pred = np.array(pred,dtype=int)
#save to file as int
np.savetxt("submission.txt", pred, delimiter="\n", fmt="%d")


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
